In [5]:
# AI Disclaimer: As per the syllabus, our group used ChatGPT to assist with aggregating and cleaning the dataset using pandas
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Load in the data
df = pd.read_csv("logistics_shipments_dataset.csv")

# Get rid of extra whitespace and clean the text
for column in ['Origin_Warehouse', 'Destination', 'Carrier']:
    df[column] = df[column].astype(str).str.replace(r"\s+", " ", regex = True).str.strip()

# Get all distinct names for origin warehouses, destination warehouses, and carriers
origins = sorted(df['Origin_Warehouse'].unique())
destinations = sorted(df['Destination'].unique())
carriers = sorted(df['Carrier'].unique())

# Map the key names to numeric index values in a dictionary
origin_index = {o: i for i, o in enumerate(origins)}
destination_index = {d: j for j, d in enumerate(destinations)}
carrier_index = {c: k for k, c in enumerate(carriers)}

# Reverse the mapping of the dictionary so index value is the key and name is the value
index_origin = {i: o for o, i in origin_index.items()}
index_destination = {j: d for d, j in destination_index.items()}
index_carrier = {k: c for c, k in carrier_index.items()}

# Length of each set for the origin, destination, and carrier
origins_len = len(origins)
dest_len = len(destinations)
carrier_len = len(carriers)

# Supply Capacity at each origin warehouse (S_i)
S = df.groupby('Origin_Warehouse')["Weight_kg"].sum().reindex(origins).fillna(0).tolist()

# Demand Capacity at each destination warehouse (D_j)
D = df.groupby('Destination')["Weight_kg"].sum().reindex(destinations).fillna(0).tolist()

# Making sure to balance supply and demand for a feasible and fair model
total_supply_kg = sum(S)
total_demand_kg = sum(D)

if total_demand_kg > 0:
    sd_ratio = total_supply_kg / total_demand_kg
    D = [d * sd_ratio for d in D]

# Carrier capacity for each carrier (C_k)
C = df.groupby("Carrier")["Weight_kg"].sum().reindex(carriers).fillna(0).tolist()

# Initialize a three-dimensional matrix for parameters c, t, p, g, and Y
co2_per_kg = [[[0 for k in carriers] for j in destinations] for i in origins]
transit_time = [[[0 for k in carriers] for j in destinations] for i in origins]
reliability = [[[0 for k in carriers] for j in destinations] for i in origins]
cost_per_kg = [[[0 for k in carriers] for j in destinations] for i in origins]
route_capacity = [[[0 for k in carriers] for j in destinations] for i in origins]

# CO2 factor per mile for different carriers (used in c_ijk calculation)
co2_factor = {'Amazon Logistics': 0.05, 'DHL': 0.15, 'FedEx': 0.08, 'LaserShip': 0.06, 'OnTrac': 0.12, 'UPS': 0.10, 'USPS': 0.11}

# Set minimum reliability rate for deliveries
R_min = 0.85

# Set the average transportation time in days for deliveries
T_average = 5

# Set the maximum C02 pollution allowed in kilograms of C02 (total supply * scaling constant)
P_max = total_supply_kg * 5

# Function to help clean missing values
def clean(x, def_value):
    if x is None:
        return def_value
    if pd.isna(x):
        return def_value
    return x

# Fill in parameters c, t, p, g, and Y for a route-carrier combination
for o in origins:
    for d in destinations:
        for c in carriers:

            # Set indexes for variables i, j, and k
            i = origin_index[o]
            j = destination_index[d]
            k = carrier_index[c]

            # Get the subset of data for that particular route-carrier combination 
            sub_df = df[(df['Origin_Warehouse'] == o) & (df['Destination'] == d) & (df['Carrier'] == c)]

            # If the subset is empty for a route-carrier combination, then fill c, t, p, g, and Y with constant average values
            if sub_df.empty:
                co2_per_kg[i][j][k] = 0.3
                transit_time[i][j][k] = 5
                reliability[i][j][k] = 0.775
                cost_per_kg[i][j][k] = 5
                route_capacity[i][j][k] = 25000
                continue


            # CO2/kilogram (c_ijk)
            avg_dist = clean(sub_df['Distance_miles'].mean(), 900)
            co2_per_kg[i][j][k] = avg_dist * co2_factor.get(c, 0.3)

            # Transit time (t_ijk)
            transit_time[i][j][k] = clean(sub_df['Transit_Days'].mean(), 5)

            # Reliability rate (p_ijk)
            delivered = sub_df[sub_df['Status'] == 'Delivered'].shape[0]
            total = sub_df.shape[0]
            if total > 0:
                reliability[i][j][k] = delivered / total
            else:
                reliability[i][j][k] = 0.775

            # Cost/kilogram (g_ijk)
            avg_cost = clean(sub_df['Cost'].mean(), 5)
            avg_weight = clean(sub_df['Weight_kg'].mean(), 1)
            cost_per_kg[i][j][k] = avg_cost / avg_weight

            # Route capacity (Y_ijk)
            avg_weight = clean(sub_df['Weight_kg'].mean(), 0)
            route_capacity[i][j][k] = max(avg_weight, 25000)

# Initialize model
m = gp.Model("Shipping")
m.Params.LogToConsole = 0

# Decision variable (x_ijk) for kilograms to be shipped on a route using a specific carrier
x = m.addVars(origins_len, dest_len, carrier_len, lb = 0, name = "x")

# Objective Function
m.setObjective(gp.quicksum(cost_per_kg[i][j][k] * x[i,j,k] for i in range(origins_len) for j in range(dest_len) for k in range(carrier_len)), GRB.MINIMIZE)

# Supply Constraint
for i in range(origins_len):
    m.addConstr(gp.quicksum(x[i,j,k] for j in range(dest_len) for k in range(carrier_len)) <= S[i])

# Demand Constraint
for j in range(dest_len):
    m.addConstr(gp.quicksum(x[i,j,k] for i in range(origins_len) for k in range(carrier_len)) == D[j])

# Carrier Capacity Constraint
for k in range(carrier_len):
    m.addConstr(gp.quicksum(x[i,j,k] for i in range(origins_len) for j in range(dest_len)) <= C[k])

# C02 Constraint
m.addConstr(gp.quicksum(co2_per_kg[i][j][k] * x[i,j,k] for i in range(origins_len) for j in range(dest_len) for k in range(carrier_len)) <= P_max)

# Transit Time Constraint
total_kg = gp.quicksum(x[i,j,k] for i in range(origins_len) for j in range(dest_len) for k in range(carrier_len))

m.addConstr(gp.quicksum(transit_time[i][j][k] * x[i,j,k] for i in range(origins_len) for j in range(dest_len) for k in range(carrier_len)) <= T_average * total_kg)

# Reliability Constraint
m.addConstr(gp.quicksum(reliability[i][j][k] * x[i,j,k] for i in range(origins_len) for j in range(dest_len) for k in range(carrier_len)) >= R_min * total_kg)

# Route Capacity Constraint
for i in range(origins_len):
    for j in range(dest_len):
        for k in range(carrier_len):
            m.addConstr(x[i,j,k] <= route_capacity[i][j][k])

# Optimize model
m.optimize()

# Print results for optimal shipping routes and weights shipped for chosen routes
print("\nOptimal Shipping Cost:", m.objVal)
print("\nShipment Plan:")
for i in range(origins_len):
    for j in range(dest_len):
        for k in range(carrier_len):
            cost = x[i,j,k].x
            if cost > 0:
                print(f"{index_origin[i]} to {index_destination[j]} using {index_carrier[k]}: {cost:.2f} kg")

# Print the reduced costs
print("\nReduced Costs:")
for i in range(origins_len):
    for j in range(dest_len):
        for k in range(carrier_len):
            rc = x[i,j,k].RC
            print(f"{index_origin[i]} to {index_destination[j]} using {index_carrier[k]}: Reduced Cost = {rc:.4f}")

Set parameter LogToConsole to value 0

Optimal Shipping Cost: 216003.9765173699

Shipment Plan:
Warehouse_ATL to Denver using OnTrac: 1177.88 kg
Warehouse_ATL to Houston using Amazon Logistics: 1508.37 kg
Warehouse_ATL to Portland using DHL: 616.95 kg
Warehouse_ATL to Seattle using USPS: 3046.90 kg
Warehouse_BOS to Boston using UPS: 2272.23 kg
Warehouse_BOS to New York using USPS: 2876.31 kg
Warehouse_BOS to Phoenix using FedEx: 4767.70 kg
Warehouse_BOS to Portland using FedEx: 46.36 kg
Warehouse_CHI to Chicago using LaserShip: 4343.00 kg
Warehouse_CHI to Detroit using DHL: 1256.21 kg
Warehouse_CHI to New York using UPS: 79.09 kg
Warehouse_DEN to Miami using LaserShip: 3161.40 kg
Warehouse_DEN to Portland using Amazon Logistics: 1922.80 kg
Warehouse_HOU to Denver using USPS: 794.30 kg
Warehouse_HOU to Los Angeles using FedEx: 3379.49 kg
Warehouse_HOU to Portland using OnTrac: 1391.89 kg
Warehouse_HOU to San Francisco using DHL: 148.62 kg
Warehouse_LA to Atlanta using OnTrac: 3222.54 kg